# Trabalho Final

In [1]:
import glob
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.debugger import Pdb
from sklearn.neural_network import MLPClassifier

%matplotlib inline

Primeiramente definimos as funções que serão utilizadas para treinamento da rede neural.

In [2]:
def extract_feature(file_name):
    '''
    Extrai as features que serão utilizadas como entrada na rede neural
    '''
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0) 
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    sr=sample_rate).T,axis=0)
    return mfccs,chroma,mel,contrast,tonnetz

def parse_audio_files(parent_dir,sub_dirs,file_ext="*.wav"):
    '''
    Obtem o arquivo de audio do diretorio, extrai as features e grava em um array
    '''
    features, labels = np.empty((0,193)), np.empty(0)
    for label, sub_dir in enumerate(sub_dirs):
        for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            try:
              mfccs, chroma, mel, contrast,tonnetz = extract_feature(fn)
            except Exception as e:
              print("Error encountered while parsing file: {}".format(fn))
              continue
            ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
            features = np.vstack([features,ext_features])
            labels = np.append(labels, fn.split('/')[2].split('-')[1])
    return np.array(features), np.array(labels, dtype = np.int)

def one_hot_encode(labels):
    Pdb.set_trace() #debug
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [3]:
parent_dir = 'audio'
tr_sub_dirs = ["fold1"]
ts_sub_dirs = ["fold6"]
tr_features, tr_labels = parse_audio_files(parent_dir,tr_sub_dirs)
ts_features, ts_labels = parse_audio_files(parent_dir,ts_sub_dirs)

tr_labels = one_hot_encode(tr_labels)
ts_labels = one_hot_encode(ts_labels)

IndexError: list index out of range

In [41]:
a = np.array([[1,2,3],[4,5,6],[7,8,9]])
b = np.array([10,10,10])
c = np.array([np.append(a[i],b[i]) for i in range(len(a))])

In [42]:
c

array([[ 1,  2,  3, 10],
       [ 4,  5,  6, 10],
       [ 7,  8,  9, 10]])